In [ ]:
import time
import requests
import urllib.request
from bs4 import BeautifulSoup

In [ ]:
'''
#get url from the home page
#get all the news' url on n pages
n = 100
url_list = []
for i in range(1, n+1):
    #get page i
    #businesshome = 'https://www.reuters.com/news/archive/businessnews?view=page&page={}&pageSize=10'.format(i)
    techhome = 'https://www.reuters.com/news/archive/technologynews?view=page&page={}&pageSize=10'.format(i)
    #newspage = 'https://www.reuters.com/news/archive/businessnews?view=page&page={}&pageSize=10'.format(i)
    html0 = urllib.request.urlopen(techhome).read()
    soup0 = BeautifulSoup(html0)
    #print(soup0.prettify())
    
    #get all the url on page i
    l = soup0.find_all('div', attrs={"class":"story-content"})
    for li in l:
        #print(li.a['href'])
        url_list.append('https://www.reuters.com'+li.a['href'])

#url_list = list(set(url_list))   
'''

In [ ]:
import csv
rows = []
with open('sp500.csv') as csvfile:
    spamreader = csv.reader(csvfile)
    
    for row in spamreader:
        rows.append(row)
rows

In [ ]:
#pair bloomberg ticker with ruters instrument code

n=0

for row in rows:
    #print(row[0])
    htmla = urllib.request.urlopen('https://www.reuters.com/companies/'+row[0]+'.N').read()
    #print('https://www.reuters.com/companies/'+row[0]+'.N')
    soupa = BeautifulSoup(htmla)
    url_lista = []
    lines = soupa.find_all('a', attrs={"class":"TextLabel__text-label___3oCVw TextLabel__black-to-orange___23uc0 TextLabel__medium___t9PWg MarketStoryItem-headline-2cgfz"})
    for line in lines:
        url_lista.append(line['href'])
    #print(url_lista)
    if url_lista != []:
        row[0] += '.N'
        n+=1
        print(n, row[0])
    
    else:
        htmlb = urllib.request.urlopen('https://www.reuters.com/companies/'+row[0]+'.O').read()
        #print('https://www.reuters.com/companies/'+row[0]+'.O')
        soupb = BeautifulSoup(htmlb)
        url_listb = []
        lines = soupb.find_all('a', attrs={"class":"TextLabel__text-label___3oCVw TextLabel__black-to-orange___23uc0 TextLabel__medium___t9PWg MarketStoryItem-headline-2cgfz"})
        for line in lines:
            url_listb.append(line['href'])
        
        if url_listb != []:
            row[0] += '.O'
            n+=1
            print(n, row[0])
        else:
            n+=1
            print(n, row[0], 'Wrong')
    
    
    #soup5 = BeautifulSoup(html5)
    #print(soup5.prettify())
    

In [ ]:
data = []
with open('sp500.csv') as csvfile:
    spamreader = csv.reader(csvfile)
    
    for d in spamreader:
        data.append(d)
data

In [ ]:
ticker_list = []
for a, b in zip(data, rows):
    ticker_list.append([a[0], b[0]])
    
ticker_list

In [ ]:
import os
import time
from selenium import webdriver

driver=webdriver.Chrome(executable_path='/Users/xiaobai/Downloads/chromedriver')

#stop every time after scroll down
def execute_times(times):
    for i in range(times):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

for i in range(72, 73):
    
    #create a folder for ticker i
    path = '/Users/xiaobai/Desktop/New_data/{}'.format(ticker_list[i][0])
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)

    #get the company's page and roll down
    roll = 0
    driver.get("https://www.reuters.com/companies/"+ticker_list[i][1])
    html_ = driver.page_source
    soup_ = BeautifulSoup(html_)
    #ago = soup_.find_all('time', attrs={"class":"TextLabel__text-label___3oCVw TextLabel__gray___1V4fk TextLabel__regular___2X0ym MarketStoryItem-date-H-tta"})
    execute_times(200)
    time.sleep(1)
#     while ago[-1].text != '6 years ago' and roll<13:
#         execute_times(10)
#         roll += 1
#         html_ = driver.page_source
#         soup_ = BeautifulSoup(html_)
#         ago = soup_.find_all('time', attrs={"class":"TextLabel__text-label___3oCVw TextLabel__gray___1V4fk TextLabel__regular___2X0ym MarketStoryItem-date-H-tta"})

    #get all the url on the page
    url_list = []
    html_ = driver.page_source
    soup_ = BeautifulSoup(html_)
    lines = soup_.find_all('a', attrs={"class":"TextLabel__text-label___3oCVw TextLabel__black-to-orange___23uc0 TextLabel__medium___t9PWg MarketStoryItem-headline-2cgfz"})
    for line in lines:
        url_list.append(line['href'])
        
    #get text, date in each url
    count = 0
    for url in url_list:
        try:
            html = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(html)
        except:
            continue
        
        #get ticker(s)
#         lines = soup.find_all('span')
#         tickers = []
#         for line in lines:
#             try:
#                 symbol = line['id']
#                 symbol = symbol.split('_')
#                 symbol = symbol[1]
#                 tickers.append(symbol)
#             except:
#                 pass
        #if len(tickers)>1: continue
            
        #get date
        try:
            dateline = soup.find_all('div', attrs={"class":"ArticleHeader_date"})
            date = dateline[0].text.split('/')[0][:-1]
        except:
            continue

        # get text
        links=soup.find_all('p')
        article=[]
        for link in links:
            article.append(link.text)
            text = ''.join(article)
        
        # ouput news to each file
        outfile=open(path+'/'+str(count)+'_'+date+'.txt','w')
        outfile.write(date)
        outfile.write('\n')
        outfile.write(text)
        outfile.write('\n')
        outfile.close()
        
        count += 1
    
    